In [1]:
from pathlib import Path
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from model.dali_pipe import dali_generator
from model.resnet import Resnet50
from model.lars import LARS
from model.scheduler import WarmupExponentialDecay
import horovod.tensorflow as hvd
import tensorflow_addons as tfa

In [2]:
hvd.init()

data_dir = Path('/home/ubuntu/shared_workspace/data/imagenet/')
index_dir = Path('/home/ubuntu/shared_workspace/data/imagenet_index/')
train_files = [i.as_posix() for i in data_dir.glob('*1024')]
train_index = [i.as_posix() for i in index_dir.glob('*1024')]

num_epochs = 70
global_batch = 512
per_gpu_batch = global_batch//hvd.size()
image_count = 1282048
steps_per_epoch = image_count//global_batch
learning_rate = 0.01*global_batch/256
scaled_rate = 0.1*global_batch/256

'''tf.keras.backend.set_floatx('float16')
tf.keras.backend.set_epsilon(1e-4)'''
tf.config.optimizer.set_jit(True)
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')

In [5]:
scheduler = WarmupExponentialDecay(learning_rate, 
                                   scaled_rate, steps_per_epoch*5, steps_per_epoch*30, 1e-5)
train_tf = dali_generator(train_files, train_index, 
                          per_gpu_batch, num_threads=8, device_id=hvd.rank(), total_devices=hvd.size())
model = tf.keras.applications.ResNet50(weights=None, input_shape=(224, 224, 3), classes=1000)
optimizer = LARS(scheduler, use_nesterov=False, clip=False)
optimizer = tf.keras.mixed_precision.experimental.LossScaleOptimizer(optimizer, loss_scale="dynamic")
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()

In [6]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        pred = model(images, training=True)
        loss = loss_func(labels, pred)
        scaled_loss = optimizer.get_scaled_loss(loss)
    scaled_grads = tape.gradient(scaled_loss, model.trainable_variables)
    grads = optimizer.get_unscaled_gradients(scaled_grads)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

In [ ]:
for epoch in range(30):
    loss = []
    progressbar = tqdm(range(steps_per_epoch))
    for batch in progressbar:
        images, labels = next(train_tf)
        loss.append(train_step(images, labels).numpy())
        progressbar.set_description("train_loss: {0:.4f}".format(np.array(loss[-100:]).mean()))

  0%|          | 0/2504 [00:00<?, ?it/s]


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



train_loss: 5.3338:  67%|██████▋   | 1690/2504 [12:38<06:04,  2.23it/s]

In [ ]:
def validation_step(images, labels):
    with tf.device('/gpu:0'):
        pred = model(images, training=False)
        loss = loss_func(labels, pred)
        top_5_pred = tf.math.top_k(pred, k=5)[1]
        top_1_pred = tf.math.top_k(pred, k=1)[1]
        labels = tf.cast(labels, tf.int32)
    top_1 = sum([label in a_pred for label, a_pred in zip(labels, top_1_pred)])
    top_5 = sum([label in a_pred for label, a_pred in zip(labels, top_5_pred)])
    return loss, top_1, top_5

def validation(steps = 128):
    loss_tracker = []
    top_1_tracker = 0
    top_5_tracker = 0
    for _ in range(steps):
        images, labels = next(validation_tdf)
        loss, top_1, top_5 = validation_step(images, labels)
        loss_tracker.append(loss.numpy())
        top_1_tracker += top_1
        top_5_tracker += top_5
    return sum(loss_tracker)/len(loss_tracker), top_1_tracker/(steps*batch_size), top_5_tracker/(steps*batch_size)


In [7]:
validation_files = [i.as_posix() for i in data_dir.glob('*0128')]
validation_index = [i.as_posix() for i in index_dir.glob('*0128')]
validation_tdf = dali_generator(validation_files, validation_index, batch_size, num_threads=4, device_id=hvd.rank(), total_devices=hvd.size())

In [8]:
validation()

(7.04437255859375, 0.00079345703125, 0.00439453125)